# 01 - Experimentation

## Overview

### Scenario

The [Chicago Taxi Trips](https://pantheon.corp.google.com/marketplace/details/city-of-chicago-public-data/chicago-taxi-trips) dataset is one ofof [public datasets hosted with BigQuery](https://cloud.google.com/bigquery/public-data/), which includes taxi trips from 2013 to the present, reported to the City of Chicago in its role as a regulatory agency. The `taxi_trips` table size is 70.72 GB and includes more than 195 million records. The dataset includes information about the trips, like pickup and dropoff datetime and location, passengers count, miles travelled, and trip toll. 

The ML task is to predict whether a given trip will result in a tip > 20%.

### Lab flow 


1. Perform Exploratory Data Analysis and Visualization.
2. Prepare the data for the ML task in BigQuery.
3. Produce and fix the raw data schema.
4. Create a managed dataset.


In [1]:
%load_ext autoreload
%autoreload 2?

In [2]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1.types import Tensorboard, TensorboardExperiment, TensorboardRun, TensorboardTimeSeries

from google.cloud.aiplatform_v1beta1 import services
from google.cloud.aiplatform_v1beta1 import types



from google.cloud import bigquery
from google.cloud import exceptions

## Configure workspace

### Configure workspace settings

In [3]:
# GCP Project and Region
PROJECT = 'jk-mlops-dev' # Change to your project Id.
REGION = 'us-central1' # Change to your region 
GCS_BUCKET = 'gs://jk-vertex-workshop-bucket' # Change to your bucket

# BigQuery development dataset
BQ_LOCATION = 'US' # Change to your location
BQ_DATASET_NAME = 'experimentation_dataset' # Change to your BQ datasent name.

# Vertex experiment name
EXPERIMENT_NAME = 'chicago-taxi-tips-classification' 

### Create BigQuery development dataset

In [4]:
client = bigquery.Client(project=PROJECT)

dataset = None
try: 
    dataset_id = '{}.{}'.format(client.project, BQ_DATASET_NAME)
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = BQ_LOCATION
    dataset = client.create_dataset(dataset, timeout=30)
except exceptions.Conflict:
    print('Dataset already exists')

Dataset already exists


### Initialize Vertex Experiment

In [5]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=GCS_BUCKET,
    experiment=EXPERIMENT_NAME,
)

### Create Vertex TensorBoard Instance

In [7]:
from google.cloud.aiplatform.utils import TensorboardClientWithOverride

tb_client = api_client = vertex_ai.initializer.global_config.create_client(
        client_class=TensorboardClientWithOverride, location_override=REGION
)

parent = f'projects/{PROJECT}/locations/{REGION}'

In [ ]:
parent = f'projects/{PROJECT}/locations/{REGION}'
tb_proto = types.Tensorboard()
tb_proto.display_name = 'Chicago Taxi Classifier'
tb_proto.description = 'Tensorboard to track development of Chicago Taxi Classifier'

operation = tb_client.create_tensorboard(parent=parent, tensorboard=tb_proto)
tensorboard = operation.result()

In [ ]:
tensorboard.name

In [ ]:
parent = f'projects/{PROJECT}/locations/{REGION}'

tb_client.list_tensorboards(parent=parent)

In [ ]:
tb_client.get_tensorboard(name=tensorboard.name)

In [ ]:
tb_experiment_proto = TensorboardExperiment(
    display_name = 'Test experiment',
    description = 'Test experiment'
)

tb_experiment = tb_client.create_tensorboard_experiment(parent=tensorboard.name, tensorboard_experiment=tb_experiment_proto)

In [ ]:
tb_experiment.name

In [ ]:
parent

In [ ]:
tb_client.list_tensorboard_experiments(parent='projects/895222332033/locations/us-central1/tensorboards/8650218615527702528')

In [ ]:
tb_run = TensorboardRun(
    display_name = "Run 1",
    description = "Run 1"
)

In [ ]:
parent = "projects/895222332033/locations/us-central1/tensorboards/7022167350233268224/experiments/"

tb_run = tb_client.create_tensorboard_run(parent=parent, tensorboard_run_id = "tb-run-1")

In [ ]:
tb_client.delete_tensorboard_experiment(name='projects/895222332033/locations/us-central1/tensorboards/7022167350233268224/experiments/ ')

In [ ]:
tb_client.delete_tensorboard(name=tensorboard.name)

In [ ]:
dir(vertex_ai.metadata.metadata_service)

In [ ]:
vertex_ai.metadata.metadata_service.experiment_name

In [ ]:
from google.cloud.aiplatform.metadata import constants
from google.cloud.aiplatform.metadata.artifact import _Artifact
from google.cloud.aiplatform.metadata.context import _Context
from google.cloud.aiplatform.metadata.execution import _Execution
from google.cloud.aiplatform.metadata.metadata_store import _MetadataStore

In [8]:
REGION

'us-central1'

In [11]:
from google.cloud.aiplatform.utils import MetadataClientWithOverride

m_client = api_client = vertex_ai.initializer.global_config.create_client(
        client_class=MetadataClientWithOverride, location_override=REGION
)

In [12]:
parent

'projects/jk-mlops-dev/locations/us-central1'

In [13]:
m_client.list_metadata_stores(parent=parent)

ListMetadataStoresPager<metadata_stores {
  name: "projects/895222332033/locations/us-central1/metadataStores/default"
  create_time {
    seconds: 1605649253
    nanos: 234724000
  }
  update_time {
    seconds: 1619136182
    nanos: 294845000
  }
}
>

In [ ]:
m_client.get_metadata_store(name='projects/895222332033/locations/us-central1/metadataSotres/default')

## 1. Explore Chicago Taxi Dataset

In [ ]:
%%bigquery

SELECT *
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
LIMIT 10

In [ ]:
%%bigquery data

SELECT 
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek, 
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2015 
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek
;

In [ ]:
data.plot(kind='bar', x='trip_dayname', y='trip_count')

## Prepare development datasets

We add `data_split` column, where 80% of the records is set to `UNASSIGNED` while the other 20% is set to `TEST`.
This column will the custom model (or an AutoML Tables model) to split the data for learning and testing.
In the learning phase, each model will split the `UNASSIGNED` records to `train` and `eval`. The `TEST` split is will be used for the final model validation.

In [ ]:
sample_size = 1000000
year = 2020

In [ ]:
sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare,
        FORMAT_TIMESTAMP("%m-%d-%Y", trip_start_timestamp) as trip_date
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,   
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)), 
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      IF(ABS(MOD(FARM_FINGERPRINT(trip_date), 10)) < 8, 'UNASSIGNED', 'TEST') AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

In [ ]:
sql_script = sql_script.replace(
    '@PROJECT', PROJECT).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TABLE', BQ_TABLE_NAME).replace(
    '@YEAR', str(year)).replace(
    '@LIMIT', str(sample_size))

job = client.query(sql_script)
job.result()

### Analyze the development dataset with Pandas

In [ ]:
sql_script = f'''
SELECT * EXCEPT (trip_start_timestamp, data_split)
FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}
'''

sample_data = client.query(sql_script).to_dataframe()

In [ ]:
sample_data.head().T

In [ ]:
sample_data.tip_bin.value_counts()

In [ ]:
sample_data.euclidean.hist()

### Analyze the development dataset with TFDV

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=sample_data,
    stats_options=tfdv.StatsOptions(
        label_feature='tip_bin',
        weight_feature=None,
        sample_rate=1,
        num_top_values=50
    )
)

tfdv.visualize_statistics(stats)

### Save statistics

In [ ]:
stats_location = os.path.join(GCS_BUCKET, 'datasets', 'chicago_taxi', 'statistics', 'stats.pbtxt')

tfdv.write_stats_text(stats, stats_location)

### Auto-generate Data Schema 


In [ ]:
schema = tfdv.infer_schema(statistics=stats)
tfdv.display_schema(schema=schema)

In [ ]:
autogenerated_schema_location = os.path.join(GCS_BUCKET, 'datasets', 'chicago_taxi', 'autogenerated_schema', 'schema.pbtxt')

tfdv.write_schema_text(schema, autogenerated_schema_location)